# Gathering Stock Prices

In this notebook, we'll use the AlphaVantage site to download stock (and crypto) data into files that will be stored in the ./data directory.
In order for this notebook to work properly, the API key for AlphaVantage will need to be filled in.

To see what data is available check out [AlphaVantage](https://alpha-vantage.readthedocs.io/en/latest/).

This notebook requires a free API key, that can be requested on http://www.alphavantage.co/support/#api-key.

***To use this notebook you'll need to have a text file named .env with your AlphaVantage API key.*** <br>

The format of this file is <br>
ALPHA_API = 'your api key' <br>

The first few cells in this notebook are setting up necessary utilities and can be safely ignored (but must be run) unless you are interested in loading files and dealing with setting constants.

In [23]:
!pip install alpha_vantage
!pip install python-dotenv

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [30]:
# %load utils.py
import math
from dateutil.relativedelta import  relativedelta
import numpy as np
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
from pathlib import Path
import datetime as dt
from dotenv import load_dotenv
import os

load_dotenv()
ALPHA_API = os.getenv('ALPHA_API')
OPEN_PRICE = 'open'
DAY_HIGH = 'high'
DAY_LOW = 'low'
DAY_CLOSE = 'close'
ADJ_CLOSE = 'adjusted close'
DAY_VOLUME = 'volume'
DIVIDEND_AMT = 'dividend amt'
MARKET_CAP = 'market cap'
SPLIT_COEFFICIENT = 'split coef'
FALSE_VALUES = ['No', 'no', 'n', 'N','F','False', 'FALSE']
TRUE_VALUES = ['Yes', 'yes', 'y', 'Y','T','True','TRUE']
DATA_PATH = Path('./data')

def get_latest_file(file_path, filename_like, file_ext):
    """
    Find absolute path to the file with the latest timestamp given the datasource name and file extension in the path
    :param file_path: where to look for the file
    :param filename_like: the basename of the datafile.  For instance if the datasource_name is foo then the filename
          representing the latest modified file with a name like 'foo*' will be returned
    :param file_ext: the filename extension
    :return: the absolute path to the file
    """
    file_ext = file_ext if '.' in file_ext else f'.{file_ext}'
    all_files = [f for f in file_path.glob(f'{filename_like}*{file_ext}', )]
    assert len(all_files) > 0, f'Unable to find any files like {file_path / filename_like}{file_ext}'
    fname = max(all_files, key=lambda x: x.stat().st_mtime).name
    return fname

def read_latest(datasource_name, errors='raise', **kwargs):
    """
    Get the most recent version of the cleaned dataset
    :param datasource_name: name of the file to get the data from
    :param errors: if 'raise' then
    :return:
    """
    read_path = DATA_PATH
    try:
        fname = get_latest_file(read_path, datasource_name, ".csv")
        ret_df = pd.read_csv(read_path / fname, index_col=0, infer_datetime_format=True, true_values=TRUE_VALUES,
                       false_values=FALSE_VALUES, **kwargs)
    except AssertionError:
        ret_df = None
        if errors != 'ignore':
            raise
    finally:
        return ret_df
    
def write_data(df, datasource_name, with_ts=True, **kwargs):
    """
    Export the dataset to a file
    :param df: the dataset to write
    :param datasource_name: the base filename to write
    :param with_ts: if True, then append the year, month, day and hour to the filename to be written
                    else append the suffix 'latest' to the basename
    :param idx: the name of the index or the column number
    :return: the name of the file written
    """
    NOW = dt.datetime.now()
    fn = make_ts_filename(DATA_PATH, src_name=datasource_name, suffix='.csv', with_ts=with_ts)

    if 'float_format' not in kwargs.keys():
        kwargs['float_format'] = '%.3f'
    df.to_csv(fn, **kwargs)
    return fn


def make_ts_filename(dir_name, src_name, suffix, with_ts=True):
    """
    Get a path with the filename specified by src_name with or without a timestamp, in the appropriate directory
    :param dir_name:
    :param src_name:
    :param suffix:
    :param with_ts:
    :return:
    """
    NOW = dt.datetime.now()
    TODAY = dt.datetime.today()
    #If the date isn't detailed enough, then uncomment the next line which will add hours, min and seconds
    #filename_suffix = f'{TODAY.month:02d}{TODAY.day:02d}_{NOW.hour:02d}{NOW.minute:02}{NOW.second:02d}' \
    filename_suffix = f'{TODAY.month:02d}{TODAY.day:02d}' \
        if with_ts else "latest"
    fn = f'{src_name}_{filename_suffix}'
    suffix = suffix if '.' in suffix else f'.{suffix}'
    filename = (dir_name / fn).with_suffix(suffix)
    return filename

# Getting the data

Okay, well now we are set up to grab the data from AlphaVantage.  This is the function that will grab the data for us.  We can get just the last 100 days (default) or we can get up to the entire 20 year history.  If we do get the 20 year history, the data is stored so when we refresh it only grabs the last 100 days to refresh the data.

In [31]:
def GetHistoricalTicker(ticker, full=False, save_data=False):
    '''
    This will gather the standard data for a stock from AlphaVantage, including
    open, close, adjusted close, volume, dividend data, and split coefficient.  Other information is available
    using other API endpoints.
    :param ticker: Ticker symbol for the security required
    :param full: If True, then get up to 20 years worth of data and will store the data to a .csv file so that we can avoid having
    to go back to the API again.  If False, it will only gather the last 100 days and data will not be saved.
    :param save_data: This value is only relevant if full=False.  If True, we'll write the results to a file using the current date
    :return: A pandas dataframe of the information from AlphaVantage
    '''

    # Using AlphaVantage https://alpha-vantage.readthedocs.io/en/latest/
    # As of now, this just gets the last 100 days if I need more I'll have to do specify
    #  outputsize = 'full' which would bring back everything.  In that case, I'd likely try to load it from
    #  the CSV file instead and then merge together
    ts = TimeSeries(ALPHA_API, output_format='pandas')
    meta = None
    if full:
        historical = read_latest(ticker, errors='ignore')
        if historical is None:
            historical, meta = ts.get_daily_adjusted(symbol=ticker,outputsize='full')
            historical.rename(columns={'1. open':OPEN_PRICE, '2. high':DAY_HIGH,
                                       '3. low':DAY_LOW, '4. close':DAY_CLOSE,
                                       '5. adjusted close':ADJ_CLOSE, '6. volume':DAY_VOLUME,
                                       '7. dividend amount':DIVIDEND_AMT, '8. split coefficient':SPLIT_COEFFICIENT}
                              ,inplace=True)
            write_data(historical,ticker)
            ticker_data=historical
        else:
            # If we have old data, then we'll go back to the source and get new data
            if relativedelta(pd.to_datetime(historical.index[-1]), TODAY).days > 0:
                last_100, meta = ts.get_daily_adjusted(symbol=ticker,outputsize='compact')
                last_100.rename(columns={'1. open': OPEN_PRICE, '2. high': DAY_HIGH,
                                           '3. low': DAY_LOW, '4. close': DAY_CLOSE,
                                           '5. adjusted close': ADJ_CLOSE, '6. volume': DAY_VOLUME,
                                           '7. dividend amount': DIVIDEND_AMT, '8. split coefficient': SPLIT_COEFFICIENT}
                                  , inplace=True)
                ticker_data = historical.append(last_100)
                ticker_data.drop_duplicates(inplace=True)
                write_data(ticker_data,ticker)
            else:
                ticker_data = historical
    else:
        last_100, meta = ts.get_daily_adjusted(symbol=ticker,outputsize='compact')
        last_100.rename(columns={'1. open': OPEN_PRICE, '2. high': DAY_HIGH,
                                           '3. low': DAY_LOW, '4. close': DAY_CLOSE,
                                           '5. adjusted close': ADJ_CLOSE, '6. volume': DAY_VOLUME,
                                           '7. dividend amount': DIVIDEND_AMT, '8. split coefficient': SPLIT_COEFFICIENT},
                        inplace=True)
        if save_data:
            write_data(last_100,ticker)
        ticker_data = last_100
    return ticker_data

## Getting some sample data

Here we are going to use the function to gather some data for ticker symbol 'AAPL'.  While we are only getting 100 trading days worth, the file will be saved and if we ask for it again, we'll add the days we are missing since we last asked for the data.

In [11]:
aapl_df = GetHistoricalTicker('aapl',save_data=True)
aapl_df

,open,high,low,close,adjusted close,volume,dividend amt,split coef
date,,,,,,,,
2019-09-30,220.9000,224.5800,220.7900,223.97,222.7712,26318583.0,0.00,1.0
2019-10-01,225.0700,228.2200,224.2000,224.59,223.3879,36187163.0,0.00,1.0
2019-10-02,223.0600,223.5800,217.9300,218.96,217.7880,35767257.0,0.00,1.0
2019-10-03,218.4300,220.9600,215.1320,220.82,219.6381,30352686.0,0.00,1.0
2019-10-04,225.6400,227.4900,223.8900,227.01,225.7949,34755553.0,0.00,1.0
2019-10-07,226.2700,229.9300,225.8400,227.06,225.8447,30889269.0,0.00,1.0
2019-10-08,225.8200,228.0600,224.3300,224.40,223.1989,29282700.0,0.00,1.0
2019-10-09,227.0300,227.7900,225.6400,227.03,225.8148,19029424.0,0.00,1.0
2019-10-10,227.9300,230.4400,227.3000,230.09,228.8585,28962984.0,0.00,1.0


## We can also grab Crypto data

The following cells introduce a function to grab the crypto prices

In [19]:
from alpha_vantage.cryptocurrencies import CryptoCurrencies

def GetCrypto(coin):
    '''
    :return: A pandas dataframe of the information from AlphaVantage
    '''

    # Using AlphaVantage https://alpha-vantage.readthedocs.io/en/latest/
    cc = CryptoCurrencies(key=ALPHA_API, output_format='pandas')
    data, meta_data = cc.get_digital_currency_daily(symbol=coin, market='USD')

    meta = None
    historical = read_latest(coin, errors='ignore')
    if historical is None:
        data, meta_data = cc.get_digital_currency_daily(symbol=coin, market='USD')
        data.rename(columns={'1a. open (USD)':OPEN_PRICE, '2a. high (USD)':DAY_HIGH,
                             '3a. low (USD)':DAY_LOW, '4a. close (USD)':DAY_CLOSE,
                             '5. volume':DAY_VOLUME,
                             '6. market cap (USD)':MARKET_CAP}
                    ,inplace=True)
        data = data[[OPEN_PRICE, DAY_HIGH, DAY_LOW, DAY_CLOSE, DAY_VOLUME, MARKET_CAP]]
        write_data(data, coin)
        ticker_data=data
    else:
        # If we have old data, then we'll go back to the source and get new data
        if relativedelta(pd.to_datetime(historical.index[-1]), TODAY).days > 0:
            last_100, meta = cc.get_digital_currency_daily(symbol=coin, market='USD')
            last_100.rename(columns={'1a. open (USD)':OPEN_PRICE, '2a. high (USD)':DAY_HIGH,
                                     '3a. low (USD)':DAY_LOW, '4a. close (USD)':DAY_CLOSE,
                                     '5. volume':DAY_VOLUME,
                                     '6. market cap (USD)':MARKET_CAP}
                            ,inplace=True)
            last_100 = last_100[[OPEN_PRICE, DAY_HIGH, DAY_LOW, DAY_CLOSE, DAY_VOLUME, MARKET_CAP]]
            ticker_data = historical.append(last_100)
            ticker_data.drop_duplicates(inplace=True)
            write_data(ticker_data, coin)
        else:
            ticker_data = historical
    return ticker_data

In [20]:
# Here we are getting the Bitcoin prices
GetCrypto('BTC')

,open,high,low,close,volume,market cap
date,,,,,,
2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377,795.150377
2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264,1199.888264
2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763,381.309763
2017-08-20,4120.98,4211.08,4032.62,4086.29,467.083022,467.083022
2017-08-21,4069.13,4119.62,3911.79,4016.00,691.743060,691.743060
2017-08-22,4016.00,4104.82,3400.00,4040.00,966.684858,966.684858
2017-08-23,4040.00,4265.80,4013.89,4114.01,1001.136565,1001.136565
2017-08-24,4147.00,4371.68,4085.01,4316.01,787.418753,787.418753
2017-08-25,4316.01,4453.91,4247.48,4280.68,573.612740,573.612740
